# Datenaufbereitung

Als erstes werden notwendige <span style="color:seagreen">packages</span> installiert und <span style="color:coral">csv</span> Dateien von Netflix und Disney+ eingelesen und in <span style="color:lightblue">df_list</span> gespeichert für eine schnellere Datenbereinigung und weniger Codezeilen und für das Zusammenführen.

# Inhaltsverzeichnis

- [Cleaning](#cleaning)  : [Für Beide](##für-beide) & [Netflix](##netflix) & [Disney+](##disney)
- [Speichern](#1.-speichern)
- [Zusammenführen und Speichern](#zusammenführen-und-speichern)

In [2]:
import pandas as pd
import numpy as np
import os
import plotly as pl
import pycountry

notebook_dir = os.path.dirname(os.path.abspath("__file__"))
dataset_dir = os.path.join(notebook_dir, '../1_Datenset/ursprüngliche')

disney_df = pd.read_csv(os.path.join(dataset_dir, 'disney_plus_titles.csv'), sep=',')
netflix_df = pd.read_csv(os.path.join(dataset_dir, 'netflix_titles.csv'), sep=',')

df_list = [
    ('disney_df', disney_df),
    ('netflix_df', netflix_df)
]

## Cleaning

### Für Beide

1. Leere Zellen in der Spalte: Director, ersetzten mit: unknown
2. Leere Zellen in der Spalte: Cast, ersetzten mit: unknown
3. Leere Zellen in der Spalte: Country, ersetzten mit: unknown

In [3]:
print('Nullwerte pro Spalte: ')
for name, df in df_list:
    df['director'] = df['director'].fillna('unknown')
    df['cast'] = df['cast'].fillna('unknown')
    df['country'] = df['country'].fillna('unknown')

    print(name)
    print(df.isna().sum())


Nullwerte pro Spalte: 
disney_df
show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      3
release_year    0
rating          3
duration        0
listed_in       0
description     0
dtype: int64
netflix_df
show_id          0
type             0
title            0
director         0
cast             0
country          0
date_added      10
release_year     0
rating           4
duration         3
listed_in        0
description      0
dtype: int64


4. Zeilen, wo leere Zellen in rating und duration sind, löschen
5. Spalten die nicht benötigt werden löschen: date_added, release_year

In [4]:
for name, df in df_list:
    print(name)
    print('Shape davor:')
    print(df.shape)

    df.dropna(subset=['duration', 'rating'], inplace=True)

    print('Shape danach:')
    print(df.shape)
    
    df.drop(columns=['date_added', 'release_year'], inplace=True)
    


disney_df
Shape davor:
(1450, 12)
Shape danach:
(1447, 12)
netflix_df
Shape davor:
(8807, 12)
Shape danach:
(8800, 12)


6. Länder anschauen und anschließend splitten
    - wieso gibt es so viele?
    - sind sie plausibel?

-> 'United States, Ghana, Burkina Faso, United Kingdom, Germany, Ethiopia' das hier wird als ein Land gewertet, welche falsch ist

#### Netflix

In [5]:
netflix_df['country'].unique()


array(['United States', 'South Africa', 'unknown', 'India',
       'United States, Ghana, Burkina Faso, United Kingdom, Germany, Ethiopia',
       'United Kingdom', 'Germany, Czech Republic', 'Mexico', 'Turkey',
       'Australia', 'United States, India, France', 'Finland',
       'China, Canada, United States',
       'South Africa, United States, Japan', 'Nigeria', 'Japan',
       'Spain, United States', 'France', 'Belgium',
       'United Kingdom, United States', 'United States, United Kingdom',
       'France, United States', 'South Korea', 'Spain',
       'United States, Singapore', 'United Kingdom, Australia, France',
       'United Kingdom, Australia, France, United States',
       'United States, Canada', 'Germany, United States',
       'South Africa, United States', 'United States, Mexico',
       'United States, Italy, France, Japan',
       'United States, Italy, Romania, United Kingdom',
       'Australia, United States', 'Argentina, Venezuela',
       'United States, Unit

In [6]:
netflix_df_copy = netflix_df.copy()
netflix_df_copy['country'] = netflix_df_copy['country'].str.split(',')
netflix_df_copy['country'] = netflix_df_copy['country'].apply(lambda x: [country.strip() for country in x])
netflix_df_countries = netflix_df_copy.explode('country')
netflix_movies_per_country = netflix_df_countries['country'].value_counts()
print(netflix_movies_per_country)

country
United States     3687
India             1046
unknown            830
United Kingdom     806
Canada             445
                  ... 
Sudan                1
Panama               1
Uganda               1
East Germany         1
Montenegro           1
Name: count, Length: 124, dtype: int64


In [7]:
netflix_df_copy_2 = netflix_df.copy()
netflix_df_copy_2['director'] = netflix_df_copy_2['director'].str.split(',')
netflix_df_copy_2['director'] = netflix_df_copy_2['director'].apply(lambda x: [director.strip() for director in x])
netflix_df_director = netflix_df_copy_2.explode('director')
netflix_movies_per_director = netflix_df_director['director'].value_counts()
print(netflix_movies_per_director)

director
unknown                   2631
Rajiv Chilaka               22
Jan Suter                   21
Raúl Campos                 19
Suhas Kadav                 16
                          ... 
Robert Cullen                1
Kirsten Johnson              1
Lawrence Kasdan              1
Milla Harrison-Hansley       1
Alicky Sussman               1
Name: count, Length: 4992, dtype: int64


In [8]:
def is_valid_country(country_name):
    if pd.isna(country_name) or country_name.lower() in ['unknown'] + ['russia']:
        return True
    try:
        pycountry.countries.lookup(country_name)
        return True
    except LookupError:
        return False

def is_valid_country(country_name):
    valid_countries = ['turkey', 'russia', 'palestine', 'vatican city', 'soviet union']
    if pd.isna(country_name) or country_name.lower() in ['unknown'] + valid_countries:
        return True
    if country_name.lower() == 'soviet union':
        country_name = 'russia'
    if country_name.lower() in ['west germany', 'east germany']:
        country_name = 'germany'
    try:
        pycountry.countries.lookup(country_name)
        return True
    except LookupError:
        return False


invalid_countries = netflix_df['country'].apply(lambda x: [country.strip() for country in str(x).split(',') if country.strip() != 'unknown' and not is_valid_country(country.strip())])
print('Nicht plasible Länder')

print(invalid_countries[invalid_countries.apply(len) > 0])



Nicht plasible Länder
193     []
365     []
1192    []
2224    []
4653    []
5925    []
7007    []
Name: country, dtype: object


In [9]:
print(netflix_df.loc[193, ['show_id', 'title', 'country']])
print('--------------------------------------------------------------------')
print(netflix_df.loc[365, ['show_id', 'title', 'country']])
print('--------------------------------------------------------------------')
print(netflix_df.loc[1192, ['show_id', 'title', 'country']])
print('--------------------------------------------------------------------')
print(netflix_df.loc[2224, ['show_id', 'title', 'country']])
print('--------------------------------------------------------------------')
print(netflix_df.loc[4653, ['show_id', 'title', 'country']])
print('--------------------------------------------------------------------')
print(netflix_df.loc[5925, ['show_id', 'title', 'country']])
print('--------------------------------------------------------------------')
print(netflix_df.loc[7007, ['show_id', 'title', 'country']])

show_id             s194
title               D.P.
country    , South Korea
Name: 193, dtype: object
--------------------------------------------------------------------
show_id                 s366
title        Eyes of a Thief
country    , France, Algeria
Name: 365, dtype: object
--------------------------------------------------------------------
show_id              s1193
title          The Present
country    United Kingdom,
Name: 1192, dtype: object
--------------------------------------------------------------------
show_id                                     s2225
title                                       Funan
country    France, Belgium, Luxembourg, Cambodia,
Name: 2224, dtype: object
--------------------------------------------------------------------
show_id             s4654
title         City of Joy
country    United States,
Name: 4653, dtype: object
--------------------------------------------------------------------
show_id              s5926
title              Virunga
co

In [10]:
#Länder umändern, sie pausible machen
netflix_df.loc[365, 'country'] = netflix_df.loc[365, 'country'].replace(', France, Algeria', 'France, Algeria')
netflix_df.loc[4653,'country'] = netflix_df.loc[4563, 'country'].replace('United States,', 'United States')
netflix_df.loc[5925, 'country'] = netflix_df.loc[5925, 'country'].replace('United Kingdom,', 'United Kingdom')
netflix_df.loc[7007, 'country'] = netflix_df.loc[7007, 'country'].replace('Poland,', 'Poland')
netflix_df.loc[1192, 'country'] = 'France, Belgium, Luxembourg, Cambodia'
netflix_df.loc[193, 'country'] = netflix_df.loc[193, 'country'].replace(', South Korea', 'South Korea')
netflix_df.loc[2224,'country'] = 'United States'

In [11]:
invalid_countries = netflix_df['country'].apply(lambda x: [country.strip() for country in str(x).split(',') if country.strip() != 'unknown' and not is_valid_country(country.strip())])
print('Nicht plasible Länder:')
if len(invalid_countries[invalid_countries.apply(len) > 0]) == 0:
    print('Alle Länder sind plausibel')
else:
    print(invalid_countries[invalid_countries.apply(len) > 0])


Nicht plasible Länder:
Alle Länder sind plausibel


#### Disney+

##### Länder

In [12]:
disney_df['country'].unique()

array(['unknown', 'United States', 'United States, Canada',
       'United States, Australia', 'Canada',
       'United States, United Kingdom', 'United States, South Korea',
       'Ireland, United States, Canada, United Kingdom, Denmark, Spain, Poland, Hungary',
       'France, United Kingdom', 'United Kingdom, Australia',
       'Ireland, United States', 'Canada, United States, France',
       'France, South Korea, Japan, United States', 'France',
       'United States, United Kingdom, Hungary', 'United States, Germany',
       'United States, United Kingdom, Australia', 'United States, India',
       'United States, Canada, United Kingdom, Singapore, Australia, Thailand',
       'Canada, United States',
       'South Korea, United States, China, Japan',
       'Australia, United Kingdom', 'United Kingdom',
       'United States, United Kingdom, South Korea',
       'United States, United Kingdom, Canada',
       'United States, Germany, United Kingdom',
       'United States, Canad

In [13]:
disney_df_copy = disney_df.copy()
disney_df_copy['country'] = disney_df_copy['country'].str.split(',')
disney_df_copy['country'] = disney_df_copy['country'].apply(lambda x: [country.strip() for country in x])
disney_df_countries = disney_df_copy.explode('country')
disney_movies_per_country = disney_df_countries['country'].value_counts()
print(disney_movies_per_country)

country
United States           1184
unknown                  216
United Kingdom           101
Canada                    77
Australia                 23
France                    22
South Korea               13
Japan                     10
China                     10
Germany                    9
Ireland                    8
Taiwan                     6
India                      5
Mexico                     4
Hong Kong                  4
Spain                      4
South Africa               3
Argentina                  3
Hungary                    3
Denmark                    3
New Zealand                3
Poland                     2
Singapore                  2
Austria                    2
Philippines                2
United Arab Emirates       2
Malaysia                   2
Brazil                     1
Switzerland                1
Tanzania                   1
Belgium                    1
Thailand                   1
Angola                     1
Panama                     1
Luxemb

In [14]:
disney_df_copy_2 = disney_df.copy()
disney_df_copy_2['director'] = disney_df_copy_2['director'].str.split(',')
disney_df_copy_2['director'] = disney_df_copy_2['director'].apply(lambda x: [director.strip() for director in x])
disney_df_director = disney_df_copy_2.explode('director')
disney_movies_per_director = disney_df_director['director'].value_counts()
print(disney_movies_per_director)

director
unknown             471
Jack Hannah          17
Wilfred Jackson      16
John Lasseter        16
Paul Hoen            16
                   ... 
Zhong Yu              1
Byron Haskin          1
Steven Lisberger      1
Jay Russell           1
Nick Castle           1
Name: count, Length: 636, dtype: int64


In [15]:
d_invalid_countries = disney_df['country'].apply(lambda x: [country.strip() for country in str(x).split(',') if country.strip() != 'unknown' and not is_valid_country(country.strip())])

print('Nicht plasible Länder')
if len(d_invalid_countries[d_invalid_countries.apply(len) > 0]) == 0:
    print('Alle Länder sind plausibel')
else:
    print(d_invalid_countries[d_invalid_countries.apply(len) > 0])


Nicht plasible Länder
Alle Länder sind plausibel


## 1. Speichern 

In [16]:
netflix_df.to_csv('../1_Datenset/erstellte/cleaned/netflix_titles_cleaned.csv', index=False)

disney_df.to_csv('../1_Datenset/erstellte/cleaned/disney_plus_titles_cleaned.csv', index=False)

## Zusammenführen und Speichern

Hier werden die 2 Datensätze zusammengeführt zu eine einzige Datensatz, dabei werden die Platforme (Netflix und Disney+) in eine extra Spalte gespeichert.

In [17]:
# Pfade zu den CSV-Dateien
netflix_cleaned_file = '../1_Datenset/erstellte/cleaned/netflix_titles_cleaned.csv'
disney_cleaned_file = '../1_Datenset/erstellte/cleaned/disney_plus_titles_cleaned.csv'

# Lade die Netflix CSV-Datei und füge eine Spalte für die Plattform hinzu
netflix_cleaned_df = pd.read_csv(netflix_cleaned_file)
netflix_cleaned_df['platform'] = 'Netflix'

# Lade die Disney+ CSV-Datei und füge eine Spalte für die Plattform hinzu
disney_cleaned_df = pd.read_csv(disney_cleaned_file)
disney_cleaned_df['platform'] = 'Disney+'

# Finde gemeinsame Titel
common_titles_cleaned = set(netflix_cleaned_df['title']).intersection(set(disney_cleaned_df['title']))

# Markiere gemeinsame Titel in beiden DataFrames
netflix_cleaned_df['platform'] = netflix_cleaned_df.apply(lambda row: 'Netflix, Disney+' if row['title'] in common_titles_cleaned else 'Netflix', axis=1)
disney_cleaned_df['platform'] = disney_cleaned_df.apply(lambda row: 'Netflix, Disney+' if row['title'] in common_titles_cleaned else 'Disney+', axis=1)

# Kombiniere die DataFrames
combined_cleaned_df = pd.concat([netflix_cleaned_df, disney_cleaned_df], ignore_index=True)

#Entferne doppelte Einträge
combined_cleaned_df = combined_cleaned_df.drop_duplicates(subset=['title'])

# Benenne die Spalte 'rating' in 'agerating' um
combined_cleaned_df.rename(columns={'rating': 'agerating'}, inplace=True)

# Gruppiere nach allen Spalten außer 'platform' und kombiniere die Plattformen
combined_cleaned_df = combined_cleaned_df.groupby(['show_id', 'type', 'title', 'director', 'cast', 'country', 'agerating', 'duration', 'listed_in', 'description'], as_index=False).agg({'platform': lambda x: ', '.join(sorted(set(x)))})

# Neuverteilung der show_id von oben nach unten in der Form 's1', 's2', ...
combined_cleaned_df['show_id'] = ['s' + str(i+1) for i in range(len(combined_cleaned_df))]

# Gib das kombinierte DataFrame aus
print(combined_cleaned_df.head())

# Speichere das kombinierte DataFrame in einer neuen CSV-Datei
combined_cleaned_df.to_csv('../1_Datenset/erstellte/fertig/fertig.csv', index=False)


  show_id   type                                             title  \
0      s1  Movie                              Dick Johnson Is Dead   
1      s2  Movie  Duck the Halls: A Mickey Mouse Christmas Special   
2      s3  Movie             A Muppets Christmas: Letters To Santa   
3      s4  Movie                                      The Starling   
4      s5  Movie                       Confessions of a Shopaholic   

                            director  \
0                    Kirsten Johnson   
1  Alonso Ramirez Ramos, Dave Wasson   
2                   Kirk R. Thatcher   
3                     Theodore Melfi   
4                         P.J. Hogan   

                                                cast        country agerating  \
0                                            unknown  United States     PG-13   
1  Chris Diamantopoulos, Tony Anselmo, Tress MacN...        unknown      TV-G   
2  Steve Whitmire, Dave Goelz, Bill Barretta, Eri...  United States         G   
3  Melissa McC

In [18]:
# Zeige die eindeutigen Werte in der Spalte 'platform'
unique_platforms = combined_cleaned_df['platform'].unique()
print(f"Einzigartige Werte in der Spalte 'platform': {unique_platforms}")

# Zähle die Anzahl der Einträge für jede Plattform
cleaned_netflix_count = combined_cleaned_df[combined_cleaned_df['platform'] == 'Netflix'].shape[0]
cleaned_disney_count = combined_cleaned_df[combined_cleaned_df['platform'] == 'Disney+'].shape[0]
cleaned_both_count = combined_cleaned_df[combined_cleaned_df['platform'] == 'Netflix, Disney+'].shape[0]

print(f"Anzahl der Netflix-Einträge: {cleaned_netflix_count}")
print(f"Anzahl der Disney+-Einträge: {cleaned_disney_count}")
print(f"Anzahl der Einträge auf beiden Plattformen: {cleaned_both_count}")


Einzigartige Werte in der Spalte 'platform': ['Netflix' 'Disney+' 'Netflix, Disney+']
Anzahl der Netflix-Einträge: 8757
Anzahl der Disney+-Einträge: 1404
Anzahl der Einträge auf beiden Plattformen: 43


## Genres vereinfachen 

zu viele vorhanden oder zum beispiel nur andere schriftweise: 'Action & Adventure' = 'Action-Adventure'

In [22]:
combined_cleaned_df_copy = combined_cleaned_df.copy()
combined_cleaned_df_copy['listed_in'] = combined_cleaned_df_copy['listed_in'].str.split(',')
combined_cleaned_df_copy['listed_in'] = combined_cleaned_df_copy['listed_in'].apply(lambda x: [genre.strip() for genre in x])
combined_cleaned_df_genres = combined_cleaned_df_copy.explode('listed_in')
combined_cleaned_df_genres['listed_in'] = combined_cleaned_df_genres['listed_in'].str.strip()
all_genres = combined_cleaned_df_genres['listed_in'].unique()
print(all_genres)
anzahl_genres = len(all_genres)
print(f"Anzahl der einzigartigen Genres: {anzahl_genres}")

['Documentaries' 'Animation' 'Family' 'Comedy' 'Musical' 'Comedies'
 'Dramas' 'Romance' 'Romantic Comedy' 'TV Comedies' 'TV Dramas'
 'International Movies' 'Thrillers' 'Action & Adventure'
 'Action-Adventure' 'International TV Shows' "Kids' TV"
 'Children & Family Movies' 'Fantasy' 'Coming of Age' 'Drama' 'Music'
 'Spanish-Language TV Shows' 'Kids' 'Animals & Nature' 'Buddy'
 'Music & Musicals' 'Concert Film' 'Sports Movies' 'Dance'
 'Independent Movies' 'Docuseries' 'Historical' 'Anime Series'
 'Horror Movies' 'Romantic Movies' 'Sci-Fi & Fantasy' 'Documentary'
 'Science & Nature TV' 'Stand-Up Comedy & Talk Shows' 'Superhero'
 'Science Fiction' 'Romantic TV Shows' 'Crime TV Shows' 'Anime Features'
 'TV Thrillers' 'Reality TV' 'Korean TV Shows' 'British TV Shows'
 'LGBTQ Movies' 'Spy/Espionage' 'Cult Movies' 'TV Horror' 'TV Mysteries'
 'Teen TV Shows' 'Anthology' 'Variety' 'Biographical' 'Sports' 'Mystery'
 'Thriller' 'Faith & Spirituality' 'Classic Movies' 'Stand-Up Comedy'
 'Crime' 'T

In [41]:
def rewrite_genres(genres):
    rewritten_genres = []
    for genre in genres:
        if genre in ['Action-Adventure', 'TV Action & Adventure', 'Superhero', 'Survival', 'Spy/Espionage']:
            rewritten_genres.append('Action & Adventure')
        elif genre in ['Documentaries', 'Docuseries']:
            rewritten_genres.append('Documentary')
        elif genre in ['Dramas', 'TV Dramas', 'Soap Opera / Melodrama']:
            rewritten_genres.append('Drama')    
        elif genre in ['Disaster', 'Crime', 'Mystery', 'TV Mysteries', 'TV Thrillers', 'Police/Cop', 'Crime TV Shows', 'Thrillers', 'Thriller']:
            rewritten_genres.append('Thriller & Krimi')
        elif genre in ['Comedies', 'TV Comedies', 'Stand-Up Comedy', 'Parody', 'Talk Show', 'Stand-Up Comedy & Talk Shows', 'Buddy']:
            rewritten_genres.append('Comedy')
        elif genre in ['Romantic Comedy', 'Romantic Movies', 'Romantic TV Shows']:
            rewritten_genres.append('Romance')
        elif genre in ['Music', 'Musical', 'Dance', 'Concert Film']:
            rewritten_genres.append('Music & Musicals')
        elif genre in ['Anime Series', 'Anime Features', 'Anime']:
            rewritten_genres.append('Animation')
        elif genre in ['Animals & Nature', 'Travel', 'Medical', 'Science & Nature TV']:
            rewritten_genres.append('Science & Nature')
        elif genre in ['Fantasy', 'Science Fiction', 'TV Sci-Fi & Fantasy']:
            rewritten_genres.append('Sci-Fi & Fantasy')
        elif genre in ['Kids', 'Family', 'Children & Family Movies', "Kids' TV"]:
            rewritten_genres.append('Children & Family')
        elif genre in ['Classic Movies', 'Classic & Cult TV', 'Cult Movies']:
            rewritten_genres.append('Classic')
        elif genre in ['Teen TV Shows', 'Coming of Age', 'LQBTQ Movies', 'Teen']:
            rewritten_genres.append('Teen & Coming of Age')
        elif genre in ['Biographical']:
            rewritten_genres.append('Historical')
        elif genre in ['TV Horror', 'Horror Movies']:
            rewritten_genres.append('Horror')
        elif genre in ['Sports Movies', 'Sports']:
            rewritten_genres.append('Sport')
        elif genre in ['British TV Shows', 'Spanish-Language TV Shows', 'Korean TV Shows', 'International Movies', 'International TV Shows']:
            rewritten_genres.append('International')
        elif genre in ['Reality TV']:
            rewritten_genres.append('Reality')
        elif genre in ['Independent Movies']:
            rewritten_genres.append('Independent')
        elif genre in ['Anthology', 'Variety', 'Game Show / Competition']:
            rewritten_genres.append('Entertainment')
        elif genre in ['Faith & Spirituality', 'Lifestyle']:
            rewritten_genres.append('Lifestyle')
        elif genre in ['TV Shows']:
            rewritten_genres.append('Series')
        else:
            rewritten_genres.append(genre)
    unique_genres = list(set(rewritten_genres))
    return ', '.join(unique_genres)

In [42]:
# Umschreibung für Gruppierung
combined_cleaned_df['listed_in'] = combined_cleaned_df['listed_in'].str.split(', ').apply(rewrite_genres)
combined_cleaned_df_genre2 = combined_cleaned_df['listed_in'].str.split(', ', expand=True).stack()
unique_genres = combined_cleaned_df_genre2.unique()
print("Namen der Genres:", unique_genres)
unique_genres = len(unique_genres)
print(f"Anzahl der einzigartigen Genres: {unique_genres}")
combined_cleaned_df_genre_counts2 = combined_cleaned_df_genre2.value_counts()
# Ausgabe der Anzahl von Genres
print("Anzahl der Genres:")
print(combined_cleaned_df_genre_counts2)

Namen der Genres: ['Documentary' 'Children & Family' 'Animation' 'Comedy' 'Music & Musicals'
 'Drama' 'Romance' 'Thriller & Krimi' 'International' 'Action & Adventure'
 'Sci-Fi & Fantasy' 'Teen & Coming of Age' 'Science & Nature' 'Sport'
 'Independent' 'Historical' 'Horror' 'Reality' 'LGBTQ Movies' 'Classic'
 'Entertainment & Lifestyle' 'Entertainment' 'Series' 'Western' 'Movies']
Anzahl der einzigartigen Genres: 25
Anzahl der Genres:
International                4268
Drama                        3319
Comedy                       3157
Children & Family            1816
Documentary                  1551
Action & Adventure           1464
Thriller & Krimi             1190
Romance                      1007
Animation                     774
Independent                   756
Sci-Fi & Fantasy              590
Music & Musicals              466
Horror                        432
Science & Nature              301
Reality                       281
Teen & Coming of Age          268
Sport            

In [43]:

# Speichere das kombinierte DataFrame in einer neuen CSV-Datei
combined_cleaned_df.to_csv('../1_Datenset/erstellte/fertig/fertig.csv', index=False)
